In [45]:
import os
import time
from dataclasses import dataclass

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tqdm.notebook import tqdm

### Set seeds for reproducibility

In [46]:
np.random.seed(42)
tf.random.set_seed(42)


### Check for GPU availability

In [47]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Using GPU: {gpus}")
else:
    print("Using CPU")


Using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [48]:
lob_features = [
    "b0p", "b1p", "b2p", "b3p", "b4p", "b5p", "b6p", "b7p", "b8p", "b9p",
    "b0q", "b1q", "b2q", "b3q", "b4q", "b5q", "b6q", "b7q", "b8q", "b9q",
    "a0p", "a1p", "a2p", "a3p", "a4p", "a5p", "a6p", "a7p", "a8p", "a9p",
    "a0q", "a1q", "a2q", "a3q", "a4q", "a5q", "a6q", "a7q", "a8q", "a9q"
]

In [49]:
@dataclass
class GANConfig:
    batch_size: int = 64
    z_dim: int = 100
    lob_dim: int = 40
    epochs: int = 100
    learning_rate_d: float = 0.0002
    learning_rate_g: float = 0.0002
    beta1: float = 0.5 
    beta2: float = 0.999
    label_smoothing: float = 0.1
    generator_target_prob: float = 0.65

config = GANConfig()

### Load and preprocess LOB data


In [50]:
file_path = "E:\DSA5204 Project\DSA5204-Group-9\lob\BTCUSDT-lob.parq"
df = pd.read_parquet(file_path, engine="pyarrow")

print(f"The legnth for the dataframe is {len(df)}")

The legnth for the dataframe is 189760


In [51]:
df = df.dropna(subset=lob_features).sample(n=10000, random_state=42)

In [52]:
scaler = MinMaxScaler()
lob_data = scaler.fit_transform(df[lob_features].values).astype(np.float32) #(5000, 40)
lob_dataset = tf.data.Dataset.from_tensor_slices(lob_data).batch(config.batch_size)


In [53]:
for batch in lob_dataset.take(1):
    print(f"Dimension of lob_dataset: {batch.shape}")

Dimension of lob_dataset: (64, 40)


In [ ]:
# Define Generator model with financial constraints
def build_generator():
    inputs = layers.Input(shape=(config.z_dim,)) #Takes in a random noise vector of size z_dim and output a tuple. Tuple ensures 1D input and not a scalar
    x = layers.Dense(1024, activation='relu')(inputs) #the first set of parentheses initializes the layer, and the second set applies it to the input tensor.
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(config.lob_dim, activation='tanh')(x)
    
    # Apply penalties using Lambda layers
    penalties = layers.Lambda(lambda x: tf.nn.softplus(-x))(x)  # Ensure non-negative prices and quantities
    bid_prices = x[:, :10]
    ask_prices = x[:, 20:30]
    bid_diff = bid_prices[:, :-1] - bid_prices[:, 1:]
    ask_diff = ask_prices[:, 1:] - ask_prices[:, :-1]
    
    # Fix: Padding to maintain shape consistency
    bid_diff_padded = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, 1]]))(bid_diff)
    ask_diff_padded = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [1, 0]]))(ask_diff)
    penalties = layers.Concatenate(axis=1)([penalties, bid_diff_padded, ask_diff_padded])
    
    #Get the max_bid and max_ask prices
    max_bid = layers.Lambda(lambda x: tf.reduce_logsumexp(x, axis=1, keepdims=True))(bid_prices)
    min_ask = layers.Lambda(lambda x: -tf.reduce_logsumexp(x, axis=1, keepdims=True))(ask_prices)
    
    penalties += layers.Lambda(lambda x: tf.nn.softplus(x))(max_bid - min_ask)
    penalty_score = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1))(penalties) #penalty score in separate variable.
    
    model = models.Model(inputs, [x, penalty_score])
    
    # Print the model output dimensions
    print(f"Generator Model Output Shapes:")
    print(f" - Generated LOB Data: {x.shape}")
    print(f" - Penalty Score (Summed Constraints): {penalties.shape}")
    
    return model

The generator outputs a model with dimension:

Generated LOB shape: (64, 40)

Penalty shape: (64,)

In [55]:
# Minibatch Discrimination code
class MinibatchDiscrimination(layers.Layer):
    """Minibatch discrimination layer to prevent mode collapse"""

    def __init__(self, num_kernels=100, dim_per_kernel=5, **kwargs):
        super(MinibatchDiscrimination, self).__init__(**kwargs)
        self.num_kernels = num_kernels
        self.dim_per_kernel = dim_per_kernel

    def build(self, input_shape):
        #Defines a trainable weight (self.kernel) of shape (input_dim, num_kernels, dim_per_kernel)
        self.input_dim = input_shape[1] #Stores the number of features in the input tensor.
        kernel_shape = (self.input_dim, self.num_kernels, self.dim_per_kernel) #Shape of the kernel tensor
        initializer = tf.random_normal_initializer(stddev=0.02)
        self.kernel = self.add_weight(
            shape=kernel_shape,
            initializer=initializer,
            name='kernel',
            trainable=True
        )
        super(MinibatchDiscrimination, self).build(input_shape)

    def call(self, inputs, **kwargs):
        #Computes a projection of inputs using self.kernel.
        #If inputs has shape (batch_size, input_dim),
        #And kernel has shape (input_dim, num_kernels, dim_per_kernel),
        #Then activation will have shape (batch_size, num_kernels, dim_per_kernel).
        #Got from dot product.
        activation = tf.tensordot(inputs, self.kernel, axes=[[1], [0]])
        
        #Expands dimensions to compare all batch samples.
        expanded_act = tf.expand_dims(activation, 3)
        transposed_act = tf.expand_dims(tf.transpose(activation, [1, 2, 0]), 0)

        diff = expanded_act - transposed_act #
        abs_diff = tf.reduce_sum(tf.abs(diff), 2) #Computes the absolute differences between all pairs of samples.

        #Apply mask to avoid comparing a sample with itself.
        batch_size = tf.shape(inputs)[0]
        mask = 1.0 - tf.eye(batch_size) 
        #Creates an identity matrix (tf.eye(batch_size)), which has 1s on the diagonal and subtracts it from 1.0. Ensures that the diagonal is 0.
        mask = tf.expand_dims(mask, 1)

        #Expands dimensions to match abs_diff.
        if mask.shape.ndims != abs_diff.shape.ndims:
            mask = tf.reshape(mask, [-1, mask.shape[1], batch_size])

        exp = tf.exp(-abs_diff) * mask
        minibatch_features = tf.reduce_sum(exp, 2)

        return tf.concat([inputs, minibatch_features], axis=1)

The final shape is (batch_size, input_dim + num_kernels).

In [56]:
# Define Discriminator model with financial constraints
def build_discriminator(use_minibatch_discrimination=False):
    inputs = layers.Input(shape=(config.lob_dim,))
    x = layers.Dense(512, activation='relu')(inputs)
    x = layers.Dense(256, activation='relu')(x)
    
    # Add Minibatch Discrimination layer
    if use_minibatch_discrimination:
        x = MinibatchDiscrimination(num_kernels=100, dim_per_kernel=5)(x)
    
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x) #ouput a rate. 
    
    # Apply penalties using Lambda layers. penalize negative prices/quantities.
    penalties = layers.Lambda(lambda x: tf.nn.softplus(-x))(inputs)

    bid_prices = inputs[:, :10]
    ask_prices = inputs[:, 20:30]
    bid_diff = bid_prices[:, :-1] - bid_prices[:, 1:]
    ask_diff = ask_prices[:, 1:] - ask_prices[:, :-1]
    
    # Fix: Padding to maintain shape consistency
    #bid_diff_padded = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, 1]]))(bid_diff)
    #ask_diff_padded = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [1, 0]]))(ask_diff)
    #penalties = layers.Concatenate(axis=1)([penalties, bid_diff_padded, ask_diff_padded])
    
    max_bid = layers.Lambda(lambda x: tf.reduce_logsumexp(x, axis=1, keepdims=True))(bid_prices)
    min_ask = layers.Lambda(lambda x: -tf.reduce_logsumexp(x, axis=1, keepdims=True))(ask_prices)
    penalties += layers.Lambda(lambda x: tf.nn.softplus(x))(max_bid - min_ask)
    
    model = models.Model(inputs, [output, penalties])
    
    # Print the model output dimensions
    print(f"Discriminator Model Output Shapes:")
    print(f" - Output (Real/Fake Probability): {output.shape}")
    print(f" - Penalties (Financial Constraints): {penalties.shape}")
    
    return model


In [57]:
# Define faulty rate computation
def compute_faulty_rate(lob_tensor):
    bid_prices = lob_tensor[:, :10]
    ask_prices = lob_tensor[:, 20:30]
    bid_quantities = lob_tensor[:, 10:20]
    ask_quantities = lob_tensor[:, 30:40]

    faulty_count = tf.reduce_sum(tf.cast(bid_prices[:, 0] >= ask_prices[:, 0], tf.float32))
    faulty_count += tf.reduce_sum(tf.cast(bid_prices[:, :-1] <= bid_prices[:, 1:], tf.float32))
    faulty_count += tf.reduce_sum(tf.cast(ask_prices[:, :-1] >= ask_prices[:, 1:], tf.float32))
    faulty_count += tf.reduce_sum(tf.cast(bid_quantities < 0, tf.float32))
    faulty_count += tf.reduce_sum(tf.cast(ask_quantities < 0, tf.float32))
    
    total_elements = tf.size(lob_tensor, out_type=tf.float32)
    faulty_rate = faulty_count / total_elements
    return faulty_rate

In [ ]:
generator = build_generator()
discriminator = build_discriminator()

optimizer_g = optimizers.Adam(learning_rate=config.learning_rate_g, beta_1=config.beta1, beta_2=config.beta2)
optimizer_d = optimizers.Adam(learning_rate=config.learning_rate_d, beta_1=config.beta1, beta_2=config.beta2)

bce_loss = tf.keras.losses.BinaryCrossentropy()

lambda_penalty = 2  # Adjust this weight based on importance of penalty term

for epoch in tqdm(range(config.epochs), desc='Training Progress'):
    for real_batch in lob_dataset:
        batch_size = tf.shape(real_batch)[0]
        real_labels = tf.ones((batch_size, 1)) * (1 - config.label_smoothing)
        fake_labels = tf.zeros((batch_size, 1))

        z = tf.random.normal((batch_size, config.z_dim))
        fake_data, fake_penalty = generator(z)

        # Train Discriminator
        with tf.GradientTape() as tape_d:
            real_pred, real_penalty = discriminator(real_batch)
            fake_pred, fake_penalty = discriminator(fake_data)

            # Binary cross-entropy loss
            loss_d = bce_loss(real_labels, real_pred) + bce_loss(fake_labels, fake_pred)

            # Add penalty term to discriminator loss
            loss_d += lambda_penalty * (tf.reduce_mean(real_penalty) + tf.reduce_mean(fake_penalty))

        grads_d = tape_d.gradient(loss_d, discriminator.trainable_variables)
        optimizer_d.apply_gradients(zip(grads_d, discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as tape_g:
            fake_data, fake_penalty = generator(z)
            fake_pred, fake_penalty = discriminator(fake_data)

            # Generator loss (trying to fool the discriminator)
            loss_g = bce_loss(real_labels, fake_pred)

            # Add penalty term to generator loss
            loss_g += lambda_penalty * tf.reduce_mean(fake_penalty)

        grads_g = tape_g.gradient(loss_g, generator.trainable_variables)
        optimizer_g.apply_gradients(zip(grads_g, generator.trainable_variables))
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{config.epochs} - Loss D: {loss_d.numpy():.4f}, Loss G: {loss_g.numpy():.4f}")

# Generate synthetic LOB data
z = tf.random.normal((10, config.z_dim))
synthetic_lob, _ = generator(z)  # Extract only the generated LOB data
synthetic_lob = synthetic_lob.numpy()
synthetic_lob = scaler.inverse_transform(synthetic_lob)
synthetic_lob_df = pd.DataFrame(synthetic_lob, columns=lob_features)


# Compute and print faulty rate
synthetic_lob_tensor = tf.convert_to_tensor(synthetic_lob, dtype=tf.float32)
faulty_rate = compute_faulty_rate(synthetic_lob_tensor)
print("Faulty Rate for Synthetic Data:", faulty_rate.numpy())

# Print synthetic LOB data
print("Synthetic LOB Data:")
print(synthetic_lob_df.head())


Generator Model Output Shapes:
 - Generated LOB Data: (None, 40)
 - Penalty Score (Summed Constraints): (None, 60)
Discriminator Model Output Shapes:
 - Output (Real/Fake Probability): (None, 1)
 - Penalties (Financial Constraints): (None, 40)


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 10/100 - Loss D: 24.4118, Loss G: 12.9762
Epoch 20/100 - Loss D: 24.8841, Loss G: 13.0898
Epoch 30/100 - Loss D: 24.2593, Loss G: 12.9902
Epoch 40/100 - Loss D: 24.6133, Loss G: 13.1201
Epoch 50/100 - Loss D: 24.1242, Loss G: 12.9613
Epoch 60/100 - Loss D: 25.0973, Loss G: 13.2964


### Testing Minibatch Discriminator

In [ ]:
generator = build_generator()
discriminator = build_discriminator(use_minibatch_discrimination=True)  

optimizer_g = optimizers.Adam(learning_rate=config.learning_rate_g, beta_1=config.beta1, beta_2=config.beta2)
optimizer_d = optimizers.Adam(learning_rate=config.learning_rate_d, beta_1=config.beta1, beta_2=config.beta2)

bce_loss = tf.keras.losses.BinaryCrossentropy()

lambda_penalty = 2  # Adjust this weight based on importance of penalty term

for epoch in tqdm(range(config.epochs), desc='Training Progress'):
    for real_batch in lob_dataset:
        batch_size = tf.shape(real_batch)[0]
        real_labels = tf.ones((batch_size, 1)) * (1 - config.label_smoothing)
        fake_labels = tf.zeros((batch_size, 1))

        z = tf.random.normal((batch_size, config.z_dim))
        fake_data, fake_penalty = generator(z)

        # Train Discriminator
        with tf.GradientTape() as tape_d:
            real_pred, real_penalty = discriminator(real_batch)
            fake_pred, fake_penalty = discriminator(fake_data)

            # Binary cross-entropy loss
            loss_d = bce_loss(real_labels, real_pred) + bce_loss(fake_labels, fake_pred)

            # Add penalty term to discriminator loss
            loss_d += lambda_penalty * (tf.reduce_mean(real_penalty) + tf.reduce_mean(fake_penalty))

        grads_d = tape_d.gradient(loss_d, discriminator.trainable_variables)
        optimizer_d.apply_gradients(zip(grads_d, discriminator.trainable_variables))

        # Train Generator
        with tf.GradientTape() as tape_g:
            fake_data, fake_penalty = generator(z)
            fake_pred, fake_penalty = discriminator(fake_data)

            # Generator loss (trying to fool the discriminator)
            loss_g = bce_loss(real_labels, fake_pred)

            # Add penalty term to generator loss
            loss_g += lambda_penalty * tf.reduce_mean(fake_penalty)

        grads_g = tape_g.gradient(loss_g, generator.trainable_variables)
        optimizer_g.apply_gradients(zip(grads_g, generator.trainable_variables))
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{config.epochs} - Loss D: {loss_d.numpy():.4f}, Loss G: {loss_g.numpy():.4f}")

# Generate synthetic LOB data
z = tf.random.normal((10, config.z_dim))
synthetic_lob, _ = generator(z)  # Extract only the generated LOB data
synthetic_lob = synthetic_lob.numpy()
synthetic_lob = scaler.inverse_transform(synthetic_lob)
synthetic_lob_df = pd.DataFrame(synthetic_lob, columns=lob_features)


# Compute and print faulty rate
synthetic_lob_tensor = tf.convert_to_tensor(synthetic_lob, dtype=tf.float32)
faulty_rate = compute_faulty_rate(synthetic_lob_tensor)
print("Faulty Rate for Synthetic Data:", faulty_rate.numpy())

# Print synthetic LOB data
print("Synthetic LOB Data:")
print(synthetic_lob_df.head())


Generator Model Output Shapes:
 - Generated LOB Data: (None, 40)
 - Penalty Score (Summed Constraints): (None, 60)
Discriminator Model Output Shapes:
 - Output (Real/Fake Probability): (None, 1)
 - Penalties (Financial Constraints): (None, 40)


Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 10/50 - Loss D: 24.8818, Loss G: 24.1941
Epoch 20/50 - Loss D: 26.3402, Loss G: 13.0394
Epoch 30/50 - Loss D: 27.0428, Loss G: 13.5552
Epoch 40/50 - Loss D: 26.1898, Loss G: 13.1127
Epoch 50/50 - Loss D: 26.7893, Loss G: 13.5987
Faulty Rate for Synthetic Data: 0.3925
Synthetic LOB Data:
            b0p           b1p           b2p           b3p           b4p  \
0  95498.570312  95498.867188  95533.718750  95521.492188  95529.054688   
1  94607.898438  94590.335938  94608.367188  94615.851562  94602.132812   
2  94665.531250  94611.617188  94627.156250  94616.945312  94606.460938   
3  95487.187500  95492.117188  95525.140625  95516.234375  95522.695312   
4  95467.515625  95474.546875  95508.531250  95501.242188  95514.187500   

            b5p           b6p           b7p           b8p           b9p  ...  \
0  95523.531250  95506.046875  95529.859375  95530.460938  95498.953125  ...   
1  94566.492188  94625.609375  94600.000000  94632.734375  94532.515625  ...   
2  94591.570312